In [1]:
# Load libraries
import pandas as pd
from sklearn.ensemble import AdaBoostClassifier
from sklearn import datasets
# Import train_test_split function
from sklearn.model_selection import train_test_split
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics

In [2]:
# Train data
X_train = pd.read_csv('x_train_smote_undersample.csv').to_numpy()
y_train = pd.read_csv('y_train_smote_undersample.csv')
y_train = y_train['LABEL']

# Test data
X_test = pd.read_csv('x_test_smote_undersample.csv').to_numpy()
y_test = pd.read_csv('y_test_smote_undersample.csv')
y_test = y_test['LABEL']

# Validation data
X_validation = pd.read_csv('x_validation_smote_undersample.csv').to_numpy()
y_validation = pd.read_csv('y_validation_smote_undersample.csv')
y_validation = y_validation['LABEL']

In [3]:
y_train

0          1
1          0
2          1
3          2
4          0
          ..
1007144    0
1007145    0
1007146    0
1007147    1
1007148    3
Name: LABEL, Length: 1007149, dtype: int64

In [4]:
# Create adaboost classifer object
abc = AdaBoostClassifier(n_estimators=50,
                         learning_rate=1)
# Train Adaboost Classifer
model = abc.fit(X_train, y_train)

#Predict the response for test dataset
y_pred = model.predict(X_test)


In [5]:
y_pred
y_test

0         0
1         0
2         0
3         0
4         0
         ..
552095    0
552096    0
552097    0
552098    0
552099    0
Name: LABEL, Length: 552100, dtype: int64

In [6]:
# Model Accuracy, how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.656143814526354


In [7]:
print('Matriz de confusion', metrics.confusion_matrix(y_test, y_pred))
print(metrics.classification_report(y_test, y_pred, target_names=['0', '1', '2', '3']))

Matriz de confusion [[249443  47186      0      0]
 [     5 112814      0      0]
 [103440      0      0      0]
 [ 39212      0      0      0]]


C:\Users\diana\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.64      0.84      0.72    296629
           1       0.71      1.00      0.83    112819
           2       0.00      0.00      0.00    103440
           3       0.00      0.00      0.00     39212

    accuracy                           0.66    552100
   macro avg       0.34      0.46      0.39    552100
weighted avg       0.49      0.66      0.56    552100



C:\Users\diana\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\diana\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [8]:
X_test

array([[0.        , 0.97856879, 0.97857939, ..., 0.        , 0.        ,
        0.9456797 ],
       [0.        , 0.00736373, 0.00736381, ..., 0.        , 0.        ,
        0.03339885],
       [0.00103334, 0.96285116, 0.96286151, ..., 0.16294643, 0.        ,
        0.9085013 ],
       ...,
       [0.00184168, 0.98103879, 0.98104927, ..., 0.        , 0.        ,
        0.96227035],
       [0.70139751, 0.98349279, 0.98345092, ..., 0.        , 0.        ,
        0.9701839 ],
       [0.        , 0.98428693, 0.98429759, ..., 0.15758929, 0.        ,
        0.97260138]])

In [9]:
# metrics.plot_roc_curve(abc, X_test, y_test)
# plt.show()